In [38]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [39]:
df = pd.read_csv('loan.csv')
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [40]:
# Removing the columns in which more than 5% is null
for i in list(df.columns):
    if (df[i].isnull().sum()/len(df))*100 > 5:
        df = df.drop(i,axis=1)

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          39717 non-null  int64  
 1   member_id                   39717 non-null  int64  
 2   loan_amnt                   39717 non-null  int64  
 3   funded_amnt                 39717 non-null  int64  
 4   funded_amnt_inv             39717 non-null  float64
 5   term                        39717 non-null  object 
 6   int_rate                    39717 non-null  object 
 7   installment                 39717 non-null  float64
 8   grade                       39717 non-null  object 
 9   sub_grade                   39717 non-null  object 
 10  emp_length                  38642 non-null  object 
 11  home_ownership              39717 non-null  object 
 12  annual_inc                  39717 non-null  float64
 13  verification_status         397

In [42]:
# Now removing the rows where ever there is null value
for i in list(df.columns):
    df = df[~df[str(i)].isnull()]

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37823 entries, 0 to 39623
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          37823 non-null  int64  
 1   member_id                   37823 non-null  int64  
 2   loan_amnt                   37823 non-null  int64  
 3   funded_amnt                 37823 non-null  int64  
 4   funded_amnt_inv             37823 non-null  float64
 5   term                        37823 non-null  object 
 6   int_rate                    37823 non-null  object 
 7   installment                 37823 non-null  float64
 8   grade                       37823 non-null  object 
 9   sub_grade                   37823 non-null  object 
 10  emp_length                  37823 non-null  object 
 11  home_ownership              37823 non-null  object 
 12  annual_inc                  37823 non-null  float64
 13  verification_status         378

In [44]:
# Removing the columns where there is only one type of entry
for i in list(df.columns):
    if len(df[i].value_counts()) == 1:
        df = df.drop(i,axis=1)

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37823 entries, 0 to 39623
Data columns (total 43 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       37823 non-null  int64  
 1   member_id                37823 non-null  int64  
 2   loan_amnt                37823 non-null  int64  
 3   funded_amnt              37823 non-null  int64  
 4   funded_amnt_inv          37823 non-null  float64
 5   term                     37823 non-null  object 
 6   int_rate                 37823 non-null  object 
 7   installment              37823 non-null  float64
 8   grade                    37823 non-null  object 
 9   sub_grade                37823 non-null  object 
 10  emp_length               37823 non-null  object 
 11  home_ownership           37823 non-null  object 
 12  annual_inc               37823 non-null  float64
 13  verification_status      37823 non-null  object 
 14  issue_d               

In [46]:
# Listing columns where there are only 2 types of entries
[item for item in df.columns if (len(df[item].value_counts()) == 2)]

['term']

In [47]:
# Listing columns where there are only 3 types of entries
[item for item in df.columns if (len(df[item].value_counts()) == 3)]

['verification_status', 'loan_status', 'pub_rec_bankruptcies']

In [48]:
# Listing the columns with object data types
[item for item in df.columns if df[item].dtype == 'O']

['term',
 'int_rate',
 'grade',
 'sub_grade',
 'emp_length',
 'home_ownership',
 'verification_status',
 'issue_d',
 'loan_status',
 'url',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'earliest_cr_line',
 'revol_util',
 'last_pymnt_d',
 'last_credit_pull_d']

In [49]:
df.last_credit_pull_d.value_counts()

May-16    9923
Apr-16    2447
Mar-16    1088
Feb-13     825
Feb-16     707
          ... 
Sep-08       3
Aug-08       3
Dec-08       3
Jun-08       1
May-08       1
Name: last_credit_pull_d, Length: 97, dtype: int64

In [50]:
# Identifying the useless columns
useless=['url','title','zip_code']
df = df.drop(useless,axis=1)

In [51]:
# rectifying the format and data type of 'revol_util' column
df.revol_util = pd.to_numeric(df.revol_util.apply(lambda x: x.replace('%','')))

In [52]:
df.last_pymnt_d

0        Jan-15
1        Apr-13
2        Jun-14
3        Jan-15
4        May-16
          ...  
39475    May-08
39551    Feb-10
39562    Aug-08
39573    May-10
39623    May-08
Name: last_pymnt_d, Length: 37823, dtype: object

In [53]:
# Fixing the last payment method date by categorizing into month and years
df['last_pymnt_d_month'] = df.last_pymnt_d.apply(lambda x: x[:3])
df['last_pymnt_d_year'] = df.last_pymnt_d.apply(lambda x: str(x[-2:]))
df = df.drop('last_pymnt_d',axis=1)

In [54]:
# vars = ['last_pymnt_d_month','last_pymnt_d_year','earliest_cr_line_month','earliest_cr_line_year',
#        'last_credit_pull_d_month','last_credit_pull_d_year','issue_d_month','issue_d_year']

In [55]:
# Fixing the earliest_cr_line by categorizing into month and years
df['earliest_cr_line_month'] = df.earliest_cr_line.apply(lambda x: x[:3])
df['earliest_cr_line_year'] = df.earliest_cr_line.apply(lambda x: str(x[-2:]))
df = df.drop('earliest_cr_line',axis=1)

In [56]:
# Fixing the last_credit_pull_d by categorizing into month and years
df['last_credit_pull_d_month'] = df.last_credit_pull_d.apply(lambda x: x[:3])
df['last_credit_pull_d_year'] = df.last_credit_pull_d.apply(lambda x: str(x[-2:]))
df = df.drop('last_credit_pull_d',axis=1)

In [57]:
# Fixing the issue_d by categorizing into month and years
df['issue_d_month'] = df.issue_d.apply(lambda x: x[:3])
df['issue_d_year'] = df.issue_d.apply(lambda x: str(x[-2:]))
df = df.drop('issue_d',axis=1)

In [58]:
# rectifying the format and data type of 'revol_util' column
df.int_rate = pd.to_numeric(df.int_rate.apply(lambda x: x.replace('%','')))

In [59]:
# Converting loan_status to suitable binary target variable
df.loan_status = df.loan_status.map(lambda x: 1 if x=='Charged Off' else 0 if x=='Fully Paid' else 'Current')

In [60]:
# Listing the columns with object data types
cat_vars = [item for item in df.columns if df[item].dtype == 'O']
cat_vars.remove('loan_status')
cat_vars

['term',
 'grade',
 'sub_grade',
 'emp_length',
 'home_ownership',
 'verification_status',
 'purpose',
 'addr_state',
 'last_pymnt_d_month',
 'last_pymnt_d_year',
 'earliest_cr_line_month',
 'earliest_cr_line_year',
 'last_credit_pull_d_month',
 'last_credit_pull_d_year',
 'issue_d_month',
 'issue_d_year']

In [61]:
# Listing the columns with numerical data types
num_vars = [item for item in df.columns if df[item].dtype != 'O']
num_vars

['id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'int_rate',
 'installment',
 'annual_inc',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_amnt',
 'pub_rec_bankruptcies']

In [62]:
# Removing the useless columns
useless_num_vars = ['id','member_id','recoveries','collection_recovery_fee','pub_rec_bankruptcies','delinq_2yrs']
df = df.drop(useless_num_vars,axis=1)

In [63]:
df = pd.get_dummies(data=df, columns=cat_vars, drop_first=True, prefix=cat_vars)

In [64]:
df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,loan_status,dti,inq_last_6mths,open_acc,...,issue_d_month_Jun,issue_d_month_Mar,issue_d_month_May,issue_d_month_Nov,issue_d_month_Oct,issue_d_month_Sep,issue_d_year_08,issue_d_year_09,issue_d_year_10,issue_d_year_11
0,5000,5000,4975.0,10.65,162.87,24000.0,0,27.65,1,3,...,0,0,0,0,0,0,0,0,0,1
1,2500,2500,2500.0,15.27,59.83,30000.0,1,1.00,5,3,...,0,0,0,0,0,0,0,0,0,1
2,2400,2400,2400.0,15.96,84.33,12252.0,0,8.72,2,2,...,0,0,0,0,0,0,0,0,0,1
3,10000,10000,10000.0,13.49,339.31,49200.0,0,20.00,1,10,...,0,0,0,0,0,0,0,0,0,1
4,3000,3000,3000.0,12.69,67.79,80000.0,Current,17.94,0,15,...,0,0,0,0,0,0,0,0,0,1


In [65]:
# Updating num_vars after removal of ueless columns
num_vars = [x for x in num_vars if x not in useless_num_vars]

In [73]:
df = df[df.loan_status != 'Current']
# test = df[df.loan_status == 'Current']

In [74]:
train,test = train_test_split(df, train_size=0.7, random_state=100)

In [75]:
scaler = StandardScaler()
train[num_vars] = scaler.fit_transform(train[num_vars])
test[num_vars] = scaler.transform(test[num_vars])
train.describe()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,inq_last_6mths,open_acc,pub_rec,...,issue_d_month_Jun,issue_d_month_Mar,issue_d_month_May,issue_d_month_Nov,issue_d_month_Oct,issue_d_month_Sep,issue_d_year_08,issue_d_year_09,issue_d_year_10,issue_d_year_11
count,2.570700e+04,2.570700e+04,2.570700e+04,2.570700e+04,2.570700e+04,2.570700e+04,2.570700e+04,2.570700e+04,2.570700e+04,2.570700e+04,...,25707.000000,25707.000000,25707.000000,25707.000000,25707.000000,25707.000000,25707.000000,25707.000000,25707.000000,25707.000000
mean,-2.764005e-18,-2.943665e-17,-8.388755e-17,-4.975209e-18,7.877414e-17,1.174702e-16,-1.663931e-16,3.717587e-17,-4.394768e-17,7.186413e-18,...,0.085541,0.067686,0.078422,0.104213,0.097289,0.090792,0.030964,0.127631,0.304936,0.536313
std,1.000019e+00,1.000019e+00,1.000019e+00,1.000019e+00,1.000019e+00,1.000019e+00,1.000019e+00,1.000019e+00,1.000019e+00,1.000019e+00,...,0.279690,0.251211,0.268840,0.305542,0.296356,0.287319,0.173224,0.333685,0.460389,0.498689
min,-1.445527e+00,-1.462523e+00,-1.488941e+00,-1.779240e+00,-1.480555e+00,-9.901632e-01,-2.009974e+00,-8.185841e-01,-1.671273e+00,-2.311860e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-7.687360e-01,-7.593340e-01,-7.762482e-01,-7.431339e-01,-7.510031e-01,-4.145926e-01,-7.655022e-01,-8.185841e-01,-7.567396e-01,-2.311860e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,-1.596245e-01,-1.827189e-01,-2.060937e-01,-3.706682e-02,-2.118233e-01,-1.419539e-01,1.285860e-02,1.224472e-01,-7.083928e-02,-2.311860e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,5.171661e-01,5.767253e-01,5.564878e-01,6.662950e-01,4.950094e-01,2.064178e-01,7.882025e-01,1.224472e-01,6.150611e-01,-2.311860e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,3.224328e+00,3.389482e+00,3.499910e+00,3.355302e+00,4.691740e+00,8.982882e+01,2.504820e+00,6.709666e+00,6.788164e+00,1.665262e+01,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [100]:
# y_train = pd.to_numeric(train['loan_status'])
# X_train = train.drop('loan_status',axis=1).iloc[:,:40]

# y_test = pd.to_numeric(test['loan_status'])
# X_test = test.drop('loan_status',axis=1).iloc[:,:40]

y_train = pd.to_numeric(train['loan_status'])
X_train = train.drop('loan_status',axis=1)

y_test = pd.to_numeric(test['loan_status'])
X_test = test.drop('loan_status',axis=1)

In [101]:
X_train.head()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,inq_last_6mths,open_acc,pub_rec,...,issue_d_month_Jun,issue_d_month_Mar,issue_d_month_May,issue_d_month_Nov,issue_d_month_Oct,issue_d_month_Sep,issue_d_year_08,issue_d_year_09,issue_d_year_10,issue_d_year_11
25531,0.720203,0.787682,0.838002,-0.337348,1.019562,-0.438403,0.608697,0.122447,0.386428,-0.231186,...,0,0,0,0,0,1,0,0,1,0
12375,0.381808,0.436087,0.506599,0.403887,0.716911,-0.566059,0.857591,-0.818584,-0.756740,-0.231186,...,1,0,0,0,0,0,0,0,0,1
8929,0.652524,0.717363,0.748915,0.403887,0.204831,0.085245,0.092807,-0.818584,0.843695,-0.231186,...,0,0,0,0,0,0,0,0,0,1
7087,0.517166,0.576725,0.649138,-0.077645,0.029589,1.902836,-1.942094,0.122447,-0.528106,-0.231186,...,0,0,0,0,0,1,0,0,0,1
23453,-0.633378,-0.618696,-0.619456,0.063027,-0.861607,-0.679658,1.411193,-0.818584,-0.528106,-0.231186,...,0,0,0,1,0,0,0,0,1,0


In [102]:
model = LogisticRegression().fit(X_train,y_train)
metrics.accuracy_score(y_test, model.predict(X_test))

0.9907424214921038

In [103]:
pred = model.predict(X_test)
pred

array([0, 0, 0, ..., 0, 0, 0])

In [104]:
pred_df = pd.DataFrame({'actual':y_test, 'predicted':pred})
pred_df = pred_df[['actual','predicted']].apply(lambda x: x.map({1:'defaulter', 0:'non-defaulter'}))
pred_df

,actual,predicted
36221,non-defaulter,non-defaulter
35960,non-defaulter,non-defaulter
37151,non-defaulter,non-defaulter
3054,non-defaulter,non-defaulter
21241,non-defaulter,non-defaulter
...,...,...
33684,non-defaulter,non-defaulter
36867,non-defaulter,non-defaulter
18894,non-defaulter,non-defaulter
12457,non-defaulter,non-defaulter
